In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb1
from xgboost import XGBClassifier

In [2]:
train_data=pd.read_csv("train_s3TEQDk.csv")
test_data=pd.read_csv("test_mSzZ8RL.csv")

In [3]:
X_train = train_data.drop(['ID','Is_Lead'], axis=1)
Y_train = train_data['Is_Lead'].values
X_test_data=test_data.drop(['ID'],axis=1)

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X_train,Y_train,test_size = 0.2,random_state=42)

In [5]:
X_train['Credit_Product'].fillna('Missing',inplace=True)
X_test['Credit_Product'].fillna('Missing',inplace=True)
X_test_data['Credit_Product'].fillna('Missing',inplace=True)

In [6]:
#Doing Label Encoding to our Categeorical features for train_data
label_encoder=LabelEncoder()
X_train['Gender']=label_encoder.fit_transform(X_train['Gender'])
X_test['Gender']=label_encoder.fit_transform(X_test['Gender'])
X_test_data['Gender']=label_encoder.fit_transform(X_test_data['Gender'])

X_train['Region_Code']=label_encoder.fit_transform(X_train['Region_Code'])
X_test['Region_Code']=label_encoder.fit_transform(X_test['Region_Code'])
X_test_data['Region_Code']=label_encoder.fit_transform(X_test_data['Region_Code'])

X_train['Occupation']=label_encoder.fit_transform(X_train['Occupation'])
X_test['Occupation']=label_encoder.fit_transform(X_test['Occupation'])
X_test_data['Occupation']=label_encoder.fit_transform(X_test_data['Occupation'])

X_train['Channel_Code']=label_encoder.fit_transform(X_train['Channel_Code'])
X_test['Channel_Code']=label_encoder.fit_transform(X_test['Channel_Code'])
X_test_data['Channel_Code']=label_encoder.fit_transform(X_test_data['Channel_Code'])

X_train['Credit_Product']=label_encoder.fit_transform(X_train['Credit_Product'])
X_test['Credit_Product']=label_encoder.fit_transform(X_test['Credit_Product'])
X_test_data['Credit_Product']=label_encoder.fit_transform(X_test_data['Credit_Product'])

X_train['Is_Active']=label_encoder.fit_transform(X_train['Is_Active'])
X_test['Is_Active']=label_encoder.fit_transform(X_test['Is_Active'])
X_test_data['Is_Active']=label_encoder.fit_transform(X_test_data['Is_Active'])

In [7]:
scaling=MinMaxScaler()
X_train[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_train[['Age','Vintage','Avg_Account_Balance']])
X_test[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_test[['Age','Vintage','Avg_Account_Balance']])
X_test_data[['Age','Vintage','Avg_Account_Balance']]=scaling.fit_transform(X_test_data[['Age','Vintage','Avg_Account_Balance']])

#### a.LogisticRegression

In [ ]:
model1 = LogisticRegression()
model1.fit(X_train, y_train)
pred_prob1 = model1.predict_proba(X_test)
auc_score1 = roc_auc_score(y_test, pred_prob1[:,1])
print(auc_score1)

0.7256740410848981


In [ ]:
predictions_on_test_data=model1.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data
submission.to_csv('submissions_hackathon/LE_mms_logitic_regression_submission.csv',index=False)

#### b.Random Forest Classifier

In [ ]:
alpha=[10,50,100,500,1000,2000]
cv_log_error_array=[]
for i in alpha:
    model2=RandomForestClassifier(n_estimators=i,random_state=24,n_jobs=-1)
    model2.fit(X_train,y_train)
    pred_prob2 = model2.predict_proba(X_test)
    auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])
    print(i)
    print("Roc_score=",auc_score2)

10
Roc_score= 0.8335547550267097
50
Roc_score= 0.8495813765275008
100
Roc_score= 0.8522954765343789
500
Roc_score= 0.8543893516289519
1000
Roc_score= 0.854936619895912
2000
Roc_score= 0.8550563587591994


In [ ]:
model2 = RandomForestClassifier(n_estimators=2000)
model2.fit(X_train, y_train)
pred_prob2 = model2.predict_proba(X_test)
auc_score2 = roc_auc_score(y_test, pred_prob2[:,1])
print(auc_score2)

0.8547065352958708


In [ ]:
predictions_on_test_data2=model2.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data2
submission.to_csv('submissions_hackathon/LE_mms_Random_Forest_submission.csv',index=False)

#### XGBoost

In [ ]:
model3 =XGBClassifier(subsample=1,n_estimators=1000,min_child_weight=7,max_depth=5,learning_rate=0.01,
                      colsample_bytree=1)
model3.fit(X_train,y_train)
pred_prob3 = model3.predict_proba(X_test)
auc_score3 = roc_auc_score(y_test, pred_prob3[:,1])
print(auc_score3)

0.8714234782309651


In [ ]:
predictions_on_test_data2=model3.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data2
submission.to_csv('submissions_hackathon/XGB_MMS_LE_submission.csv',index=False)

#### MLP

In [8]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import PReLU,Dropout
from tensorflow.keras import Sequential
from keras.callbacks import Callback

In [9]:
model = Sequential()
model.add(Dense(400, input_dim = X_train.shape[1], kernel_initializer = 'he_normal'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(200, kernel_initializer='he_normal'))
model.add(PReLU())
model.add(BatchNormalization())    
model.add(Dropout(0.2))
model.add(Dense(50, kernel_initializer = 'he_normal'))
model.add(PReLU())
model.add(BatchNormalization())    
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer = 'he_normal'))
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=[tf.keras.metrics.AUC(from_logits=True)])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               4000      
_________________________________________________________________
p_re_lu (PReLU)              (None, 400)               400       
_________________________________________________________________
batch_normalization (BatchNo (None, 400)               1600      
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 200)               200       
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               8

In [11]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20
6144/6144 [==============================] - 39s 6ms/step - loss: 0.4075 - auc: 0.8254 - val_loss: 0.3737 - val_auc: 0.8536
Epoch 2/20
6144/6144 [==============================] - 34s 6ms/step - loss: 0.3782 - auc: 0.8481 - val_loss: 0.3707 - val_auc: 0.8572
Epoch 3/20
6144/6144 [==============================] - 34s 5ms/step - loss: 0.3736 - auc: 0.8526 - val_loss: 0.3661 - val_auc: 0.8609
Epoch 4/20
6144/6144 [==============================] - 34s 5ms/step - loss: 0.3711 - auc: 0.8547 - val_loss: 0.3585 - val_auc: 0.8644
Epoch 5/20
6144/6144 [==============================] - 34s 6ms/step - loss: 0.3668 - auc: 0.8578 - val_loss: 0.3545 - val_auc: 0.8670
Epoch 6/20
6144/6144 [==============================] - 34s 5ms/step - loss: 0.3632 - auc: 0.8601 - val_loss: 0.3527 - val_auc: 0.8674
Epoch 7/20
6144/6144 [==============================] - 34s 5ms/step - loss: 0.3623 - auc: 0.8600 - val_loss: 0.3504 - val_auc: 0.8691
Epoch 8/20
6144/6144 [==============================] -

In [13]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Epoch 1/10
6144/6144 [==============================] - 35s 6ms/step - loss: 0.3571 - auc: 0.8642 - val_loss: 0.3498 - val_auc: 0.8683
Epoch 2/10
6144/6144 [==============================] - 34s 6ms/step - loss: 0.3570 - auc: 0.8642 - val_loss: 0.3500 - val_auc: 0.8690
Epoch 3/10
6144/6144 [==============================] - 35s 6ms/step - loss: 0.3570 - auc: 0.8639 - val_loss: 0.3505 - val_auc: 0.8681
Epoch 4/10
6144/6144 [==============================] - 34s 6ms/step - loss: 0.3565 - auc: 0.8642 - val_loss: 0.3503 - val_auc: 0.8696
Epoch 5/10
6144/6144 [==============================] - 37s 6ms/step - loss: 0.3565 - auc: 0.8641 - val_loss: 0.3493 - val_auc: 0.8696
Epoch 6/10
6144/6144 [==============================] - 37s 6ms/step - loss: 0.3563 - auc: 0.8643 - val_loss: 0.3501 - val_auc: 0.8696
Epoch 7/10
6144/6144 [==============================] - 34s 6ms/step - loss: 0.3561 - auc: 0.8644 - val_loss: 0.3518 - val_auc: 0.8679
Epoch 8/10
6144/6144 [==============================] -

In [14]:
predictions_on_test_data3=model.predict(X_test_data)
submission=pd.read_csv("sample_submission_eyYijxG.csv")
submission['Is_Lead']=predictions_on_test_data3
submission.to_csv('MLP2_MMS_LE_submission.csv',index=False)

In [ ]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[auroc])